In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import mountains
import matplotlib.pyplot as plt
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
mountain_detail, routes, weather_data = mountains.call_data()

In [75]:
def to_mountainrte(rte):
    route = []
    for key,value in rte.items():
        for v in value:
            route.append([key] + list(v))
    return pd.DataFrame(route)

def to_mountainwea(wea):
    weather = []
    for key,value in wea.items():
        for v in value:
            weather.append([key] + list(v))
    return pd.DataFrame(weather)


In [76]:
fin_routes = to_mountainrte(routes)
fin_routes.columns = ['Mountain','Route','Class',
                     'Snow','ElevationGain','RTMileage']
fin_routes[["ElevationGain","RTMileage"]] = fin_routes[["ElevationGain","RTMileage"]].apply(pd.to_numeric)
fin_routes

,Mountain,Route,Class,Snow,ElevationGain,RTMileage
0,Capitol Peak,Northeast Ridge,Class 4,,5300,17.00
1,Uncompahgre Peak,South Ridge,Class 2,,3000,7.50
2,Quandary Peak,East Ridge,Class 1,,3450,6.75
3,Quandary Peak,South Gully aka Cristo Couloir,Difficult Class 2,Moderate Snow,2575,2.00
4,Quandary Peak,West Ridge,Class 3,,2650,4.00
5,Quandary Peak,North Gully aka Quandary Couloir,Class 3,Steep Snow,3250,5.25
6,Huron Peak,Northwest Slopes,Class 2,,3500,6.50
7,Huron Peak,Southwest Slopes,Class 2,,3600,9.00
8,Huron Peak,North Ridge from Lulu Gulch,Class 2,,3980,8.40
9,Challenger Point,Via Challenger Point,Easy Class 3,,6250,14.50


In [77]:
fin_weather = to_mountainwea(weather_data)
fin_weather.columns = ['Mountain','Date','Temp',
                     'WindLow','WindHigh','Description']
fin_weather[["Temp","WindLow","WindLow","WindHigh"]] =\
fin_weather[["Temp","WindLow","WindLow","WindHigh"]].apply(pd.to_numeric)

In [78]:

fin_mountain = pd.DataFrame.from_dict(mountain_detail).T
fin_mountain.index.name = 'Mountain'
fin_mountain.columns = ['Elevation','Rank',
                     'Range']
fin_mountain.reset_index(level=0, inplace=True)
fin_mountain

,Mountain,Elevation,Rank,Range
0,Grays Peak,14270,9,Front
1,Torreys Peak,14267,11,Front
2,Mt. Evans,14264,14,Front
3,Longs Peak,14255,15,Front
4,Pikes Peak,14110,30,Front
5,Mt. Bierstadt,14060,38,Front
6,Quandary Peak,14265,13,Tenmile
7,Mt. Lincoln,14286,8,Mosquito
8,Mt. Cameron,14238,-1,Mosquito
9,Mt. Bross,14172,22,Mosquito


In [79]:
fin_routes.columns

Index(['Mountain', 'Route', 'Class', 'Snow', 'ElevationGain', 'RTMileage'], dtype='object')

In [80]:
# How many mountains are in each class 
# Is gain related to class
fin_routes.groupby('Class').agg({'ElevationGain': ['min', 'max','median','mean','std'],'RTMileage': ['min', 'max','median','mean','std'], 'Mountain':['nunique']})

ElevationGain                                         \
                            min   max median         mean          std   
Class                                                                    
Class 1                    2500  7600   3450  3916.666667  1528.070679   
Class 2                    2000  5800   3980  3883.220339  1041.365881   
Class 3                    1500  6100   4400  4197.222222  1383.471020   
Class 4                    3200  6200   5100  4963.444444   905.837611   
Class 5                    4400  4400   4400  4400.000000          NaN   
Difficult Class 2          2150  6500   4600  4640.294118  1125.003431   
Easy Class 3               2900  6250   4500  4537.500000  1593.410912   

                  RTMileage                                     Mountain  
                        min    max  median       mean       std  nunique  
Class                                                                     
Class 1                5.75  24.00   9.500  10.805556  5.478335        7  
Class 2                3.25  17.00   8.400   8.803390  3.157720       22  
Class 3                2.00  22.00  12.000  10.513889  4.600444       17  
Class 4                5.00  17.00  11.400  11.766667  4.422881        6  
Class 5               11.25  11.25  11.250  11.250000       NaN        1  
Difficult Class 2      2.00  17.00  13.000  11.514706  4.531548       13  
Easy Class 3           6.00  14.50   9.925  10.087500  3.734607        4

In [81]:
# What is the longest/shortest/avg route
fin_routes.groupby('Route').agg({'ElevationGain': ['min', 'max'],'RTMileage': ['min', 'max'], 'Mountain':['nunique']})

ElevationGain       RTMileage         \
                                             min   max       min    max   
Route                                                                     
"Dolly Varden Gully"                        2900  2900      4.00   4.00   
"S Gully"                                   2600  2600      9.50   9.50   
Angel of Shavano                            4600  4600      7.50   7.50   
Bell Cord Couloir                           4556  4556      8.00   8.00   
Box Creek Couloirs                          4150  4150      8.50   8.50   
Conundrum Couloir                           4400  4400     12.00  12.00   
Cross Couloir                               5600  5600     12.25  12.25   
Dead Dog Couloir                            3000  3000      6.50   6.50   
East Face                                   3300  4900      6.50  12.00   
East Ridge                                  3000  5900      5.00  14.50   
East Slope                                  2150  2150      4.00   4.00   
East Slopes                                 2650  7600      6.50  24.00   
East Slopes from Mineral Park               2900  2900      9.00   9.00   
East Slopes from Moose Creek                3100  3100      5.00   5.00   
Ellingwood Arete/Ledges                     4400  4400     11.25  11.25   
Ellingwood Ridge                            5900  5900      9.50   9.50   
Emperor Couloir                             4500  4500      9.50   9.50   
Halo Ridge                                  5210  5210     15.00  15.00   
Kelso Ridge                                 3100  3100      6.75   6.75   
Keplinger's Couloir                         5900  5900     16.00  16.00   
Keyhole Route                               5100  5100     14.50  14.50   
Kirk Couloir                                5400  5400     12.75  12.75   
Lincoln Amphitheatre                        3400  3400      4.75   4.75   
Loft Route                                  5300  5300     13.00  13.00   
Lost Rat Couloir                            3000  3000      6.50   6.50   
NW Face/Tuning Fork                         4500  4500     10.50  10.50   
North Buttress                              3200  3200      5.00   5.00   
North Couloir                               3900  3900      8.75   8.75   
North Face                                  4900  4900      8.75   8.75   
North Face Couloir                          4600  4600     13.00  13.00   
North Face Couloirs                         4500  4500      9.00   9.00   
North Face Moderate                         1500  1500      2.00   2.00   
North Face Steep                            1500  1500      2.00   2.00   
North Gully aka Quandary Couloir            3250  3250      5.25   5.25   
North Ridge                                 5315  5600     11.40  12.00   
North Ridge from Lulu Gulch                 3980  3980      8.40   8.40   
North Ridge via South Zapata Creek          5500  5500     11.60  11.60   
North Slope                                 5400  5400     12.50  12.50   
North Slopes                                3000  5100      7.50  16.00   
Northeast Ridge                             3600  6100      8.25  17.00   
Northwest Face                              3800  3800      8.50   8.50   
Northwest Gully                             3500  4500      7.25   8.25   
Northwest Ridge                             2700  6500      5.00  17.00   
Northwest Slopes                            3500  4300      6.50  14.00   
Outward Bound Couloir                       5500  5500     13.75  13.75   
South Face                                  4400  6200     12.00  17.00   
South Gully aka Cristo Couloir              2575  2575      2.00   2.00   
South Ridge                                 3000  6000      7.00  16.75   
South Slope                                 2150  3100      2.50   8.50   
South Slopes                                2900  4600      6.00  14.00   
Southeast Gully                             5400  5400      

In [82]:
# How many mountains still have snowpack - and what class are they
snow_routes = pd.DataFrame(fin_routes[fin_routes.Snow != ' '])
snow_routes.groupby('Mountain').agg({'Snow': ['count']})
snow_routes.groupby('Route').agg({'Snow': ['count']})

,Snow
,count
Route,
"""Dolly Varden Gully""",1
"""S Gully""",1
Angel of Shavano,1
Bell Cord Couloir,1
Box Creek Couloirs,1
Conundrum Couloir,1
Cross Couloir,1
Dead Dog Couloir,1


In [83]:
# Which range is the easiest to knock out (lowest avg elevation gain)
elevation = fin_routes.groupby('Mountain') \
.agg({'ElevationGain': ['min', 'max','median','mean','std','sum'], 'Route': ['count']}).round(2)
elevation.ElevationGain.sort_values(by='min')
elevation.ElevationGain.sort_values(by='max')
elevation.ElevationGain.sort_values(by='median')

,min,max,median,mean,std,sum
Mountain,,,,,,
Mt. Sherman,2100,3100,2150,2450.00,563.47,7350
Mt. Bierstadt,1500,5600,2425,2650.00,1369.83,21200
Mt. Democrat,2150,3400,2600,2631.82,413.69,28950
Culebra Peak,2700,2700,2700,2700.00,NaN,2700
Handies Peak,2500,3650,2800,2983.33,596.52,8950
Mt. Sneffels,2900,2950,2925,2925.00,35.36,5850
Quandary Peak,2575,3450,2950,2981.25,434.63,11925
Uncompahgre Peak,3000,3000,3000,3000.00,NaN,3000
Torreys Peak,3000,5500,3175,3665.00,884.45,36650


In [106]:
fin_weather.columns
range_weather = fin_mountain.merge(fin_weather,left_on='Mountain',right_on='Mountain')
agg_weather = fin_weather.groupby('Date').agg({'Temp': ['min', 'max','median','mean','std'], 'WindHigh': ['min', 'max','median','mean','std']})
#What is the average daytime temp by range
agg_weather['Temp']
agg_weather['WindHigh']
#Is there a relationship to Elevation and high winds
range_weather[["Temp","WindLow","WindLow","WindHigh","Rank","Elevation"]] = range_weather[["Temp","WindLow","WindLow","WindHigh","Rank","Elevation"]].apply(pd.to_numeric)
range_weather.groupby('Range').agg({'Elevation':['min', 'max','median','mean','std'],'Rank':['min', 'max','median','mean','std'],'Temp':['min', 'max','median','mean','std'],'WindHigh':['min', 'max','median','mean','std'],'Mountain':['nunique']})

#What is the average temp on each mtn
#Which temps are in an "enojyable" range

Elevation                                           Rank             \
               min    max   median          mean         std  min max median   
Range                                                                          
Elk          14014  14265  14092.0  14105.000000   82.418163   -1  47   24.0   
Front        14060  14270  14259.5  14204.333333   86.453018    9  38   14.5   
Mosquito     14036  14286  14172.0  14176.000000   86.122836   -1  45   22.0   
San Juan     14001  14309  14053.5  14090.071429   90.493821   -1  53   36.0   
Sangres      14037  14345  14072.5  14131.400000  108.524726    4  44   35.5   
Sawatch      14003  14433  14197.0  14224.928571  132.443263    1  52   18.5   
Tenmile      14265  14265  14265.0  14265.000000    0.000000   13  13   13.0   

                               Temp                                  WindHigh  \
               mean        std  min max median       mean        std      min   
Range                                                                           
Elk       20.142857  16.562823   40  72   50.5  52.442857  10.461296        5   
Front     19.500000  10.778981   33  57   43.0  44.750000   7.322186        9   
Mosquito  20.400000  16.144753   38  61   47.5  48.120000   8.294675       12   
San Juan  31.214286  18.478759   38  78   51.5  53.664286  10.261061        5   
Sangres   29.500000  14.377662   38  62   46.5  47.650000   7.044412       10   
Sawatch   19.285714  14.376382   38  60   46.0  48.057143   7.589060       10   
Tenmile   13.000000   0.000000   40  55   46.5  47.100000   6.740425       15   

                                         Mountain  
         max median       mean       std  nunique  
Range                                              
Elk       15   10.0  10.428571  2.517023        7  
Front     23   15.0  15.350000  3.046671        6  
Mosquito  20   15.0  15.660000  1.869301        5  
San Juan  15   10.0   9.214286  3.511104       14  
Sangres   20   15.0  16.750000  2.500000       10  
Sawatch   20   15.0  15.464286  2.235781       14  
Tenmile   20   17.5  17.400000  1.712698        1

In [107]:
range_weather
range_weather.groupby('Range').agg({'Temp': ['min', 'max','median','mean','std'], 'WindHigh': ['min', 'max','median','mean','std']})


Temp                                  WindHigh                        \
          min max median       mean        std      min max median       mean   
Range                                                                           
Elk        40  72   50.5  52.442857  10.461296        5  15   10.0  10.428571   
Front      33  57   43.0  44.750000   7.322186        9  23   15.0  15.350000   
Mosquito   38  61   47.5  48.120000   8.294675       12  20   15.0  15.660000   
San Juan   38  78   51.5  53.664286  10.261061        5  15   10.0   9.214286   
Sangres    38  62   46.5  47.650000   7.044412       10  20   15.0  16.750000   
Sawatch    38  60   46.0  48.057143   7.589060       10  20   15.0  15.464286   
Tenmile    40  55   46.5  47.100000   6.740425       15  20   17.5  17.400000   

                    
               std  
Range               
Elk       2.517023  
Front     3.046671  
Mosquito  1.869301  
San Juan  3.511104  
Sangres   2.500000  
Sawatch   2.235781  
Tenmile   1.712698